In [9]:
import numpy as np
import requests
import json
import geopandas as gpd
import pandas as pd
from zipfile import ZipFile
from io import BytesIO

In [11]:
# Download the Census Tract Geometries
url = "https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_tract_500k.zip"
response = requests.get(url)
print("Tracts Downloaded")

# Extract the Zip File to a BytesIO object (in memory extraction)
with ZipFile(BytesIO(response.content)) as zip_ref:
    zip_ref.extractall("Data/ACS_2021/Tract_Geometries")
print("Tracts Unzipped")

# Read the Shapefile
tract_file = "Data/ACS_2021/Tract_Geometries/cb_2021_us_tract_500k.shp"
tracts = gpd.read_file(tract_file)

# Create a DataFrame with Latitude and Longitude
# We use centroid to get the central point of each tract, then split its x and y to lon and lat
tracts['lon'] = tracts.geometry.centroid.x
tracts['lat'] = tracts.geometry.centroid.y
tract_locations = tracts[['GEOID', 'lon', 'lat']]

# Optionally, convert the GeoDataFrame to a Pandas DataFrame if no geometry is needed
tract_locations_df = pd.DataFrame(tract_locations)

print(tract_locations_df.head())


Tracts Downloaded
Tracts Unzipped


/var/folders/t7/ttgj0mhj6575f72vbfx0p2_40000gn/T/ipykernel_33912/342237354.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts['lon'] = tracts.geometry.centroid.x
/var/folders/t7/ttgj0mhj6575f72vbfx0p2_40000gn/T/ipykernel_33912/342237354.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tracts['lat'] = tracts.geometry.centroid.y


         GEOID         lon        lat
0  13163960300  -82.427775  33.020095
1  37109071201  -80.968056  35.459902
2  25025050200  -71.035737  42.377547
3  17031230200  -87.710082  41.911874
4  41047001607 -122.983430  44.961583


In [7]:
# Example list of latitudes and longitudes for census regions
# You'll need to replace this with your actual list/source of coordinates
regions = [
    {'lat': 37, 'lon': -92},
    # Add more regions here
]

# Replace YOUR_API_KEY with your actual NREL API key
api_key = '5NafpqgGlLOBTehX0g7wPnu6qxpYn35jGVjFBnmj'
base_url = 'https://developer.nrel.gov/api/census_rate/v3.json'

# Loop through each region and download data
for region in regions:
    params = {
        'api_key': api_key,
        'region': 'tract',
        'lat': region['lat'],
        'lon': region['lon'],
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        # Process the data or save it to a file
        # For example, saving to a JSON file:
        filename = f"data_{region['lat']}_{region['lon']}.json"
        with open(filename, 'w') as file:
            json.dump(data, file)
        print(f"Data saved for region {region['lat']}, {region['lon']}")
    else:
        print(f"Failed to download data for region {region['lat']}, {region['lon']}")

Data saved for region 37, -92
